https://github.com/mwritescode/slither-audited-smart-contracts

The bytecode was downloaded using the Web3.py library, in particular the web3.eth.getCode()

Solidity Smart Contracts that have been verified on Etherscan.io

classification of vulnerabilities

both binary and multilabel text

este trabajo: text classification

evaluated based on the accuracy

nota peresonal no poner en ppt: source codes are in Solidity

label: cleaned up version or a multi-label version

data fields: address, source_code, bytecode, slither
nota personal: una fila tardaba en compilar sta 
nota personal no poner en ppt: train, test and validation splits are only provided for those configurations that do not include "all-" in their names
nota personal no poner en ppt: a pesar de 4 columnas el tiempo de compilación era considerable. OpenAI mencionó que ven if the DataFrame has only four rows, if the data within those rows is complex, such as having many columns, long strings or complex object types, then it can take longer to execute commands on that data.

In [2]:
#!pip install web3
#!pip install datasets
#!pip install numpy==1.19.5

import numpy as np
import pandas as pd
import re
import seaborn as sns
from hexbytes import HexBytes
import matplotlib.pyplot as plt
from datasets import load_dataset
import sys
sys.path.append('MCD_DL_proyecto2')
import funciones
import re
import time

/Users/1thaliaespana1/opt/anaconda3/envs/itam/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
#extraido de Git repo mwritescode/slither-audited-smart-contracts

train_set_original = load_dataset("mwritescode/slither-audited-smart-contracts", 'big-multilabel', split='train') #verification_mode=no_checks
test_set_original = load_dataset("mwritescode/slither-audited-smart-contracts", 'big-multilabel', split='test')
val_set_original = load_dataset("mwritescode/slither-audited-smart-contracts", 'big-multilabel', split='validation')

Found cached dataset slither-audited-smart-contracts (/Users/1thaliaespana1/.cache/huggingface/datasets/mwritescode___slither-audited-smart-contracts/big-multilabel/1.1.0/4cf503b59ce9d3157914e47f6253de773b7ab828f46642685d4b470b88ca1f13)
Found cached dataset slither-audited-smart-contracts (/Users/1thaliaespana1/.cache/huggingface/datasets/mwritescode___slither-audited-smart-contracts/big-multilabel/1.1.0/4cf503b59ce9d3157914e47f6253de773b7ab828f46642685d4b470b88ca1f13)
Found cached dataset slither-audited-smart-contracts (/Users/1thaliaespana1/.cache/huggingface/datasets/mwritescode___slither-audited-smart-contracts/big-multilabel/1.1.0/4cf503b59ce9d3157914e47f6253de773b7ab828f46642685d4b470b88ca1f13)


In [5]:
#COLS_TO_REMOVE = ['source_code', 'bytecode']
#LABELS = {0:'access-control', 1:'arithmetic', 2:'other', 3:'reentrancy', 4:'safe', 5:'unchecked-calls'}

#preparar datos: 
def preparacion(df1):

    df1=pd.DataFrame(df1) #convertimos a data frame
    target=[4 in lst for lst in df1['slither']] #cambiamos variable objetivo a 1: si hay un contrato seguro, 0 no seguro-hay 4 tipos de no seguro)
    df1['slither']=pd.DataFrame(target).astype(int) #sustituimos lista original de objetivo por booleano target
    bytecod=pd.factorize(df1['bytecode']) #vemos que bytecode es diferente para cada entrada
    df1['bytecode']=bytecod[0].astype(str)
    #------------separación de columnas de source_code-------
    delim=['///','//','\n\n','\n\n\n\n','\n\n\n']
    def delimiter_sep(df,delim):
        new_cols = df['source_code'].str.split(delim, expand=True) #separamos columnas, quedan 635 en total
        df = pd.concat([df, new_cols], axis=1)
        return df
    for i in range(4):
        df1=delimiter_sep(df1,delim[i])
    column_names = ['source_col_{}'.format(jj) for jj in range(631)] #nombres de columnas 
    orig_names=['address','source_code','bytecode','slither']
    column_names=orig_names + column_names
    df1=df1.drop('address').replace("\@dev",'').replace("\n",'').replace("\*",'')
    df1.columns=column_names
    return df1.fillna("None")
 

In [79]:
#train_set=preparacion(train_set_original)
#test_set=preparacion(test_set_original)
#val_set=preparacion(val_set_original)
print(train_set)

#añadir regularización

: 

: 

In [6]:
train_set=preparacion(train_set_original)

In [ ]:
train_set

,address,source_code,bytecode,slither
0,0x006699d34AA3013605d468d2755A2Fe59A16B12B,pragma solidity 0.5.4;\n\ninterface IERC20 {\n...,0,0
1,0x00c83aeCC790e8a4453e5dD3B0B4b3680501a7A7,pragma solidity 0.6.10;\npragma experimental A...,1,0
2,0x010c5322d78c88ca18282b0a072a8913648b3038,pragma solidity 0.6.5;\npragma experimental AB...,2,1
3,0x0114622386c1a00686e594c70682d7aa0f8afa29,pragma solidity 0.6.6;\n\nlibrary Address {\n\...,3,1
4,0x01b952402442630232f32d48380a557015b7c5ec,pragma solidity 0.7.4;\n\nenum MathError { NO_...,4,0
...,...,...,...,...
79636,0x80d19b2bf1743371855732971cec24fe9dcd1417,/**\n *Submitted for verification at Etherscan...,76629,1
79637,0x810b54cdb3d8c97d857b500cdfdb5715e1d27837,contract ERC20Basic {\n uint256 public totalS...,76630,1
79638,0x8142871ce8e9b69af8b50369fde8f9959d699fa6,contract Bodies2 {\n\nstring public constant b...,76631,1
79639,0x8154094af8dc315b31c6ec746260003ee828fd93,contract LifeSet_005 {\n \n// part.I_fixe.....,76632,1


In [70]:
p=train_set_head.isna().sum().sort_values()
#len(train_set_head.columns) #sub \n @dev * 635 len

618

In [ ]:
a = 0
for i in concatenated['bytecode']:
    i = str(i)
    byte = bytes(i,'utf-8')
    if (len(byte)/3) > 1:
        width, height = closestDivisors(int(len(byte)/3))
        #print(a, width, height)
        img = Image.frombytes("RGB", (width, height), byte)
        name = 'byte_imgs/bytecode_img_'+str(a)+'.png'
        img.save(name)
        a += 1
    else:
        a += 1



1


## Referencias
- https://github.com/mwritescode/slither-audited-smart-contracts
- 